<a id="section-1"></a>
# Section 1: What Is Reasoning in Agentic Systems?

Before we can build reasoning into agents, we must understand what "reasoning" actually means in this context.

---

## 1.1 What Is Reasoning?

**Reasoning is the process by which an agent decides what to do next based on:**
1. What it knows (state)
2. What it needs to know (goals)
3. What it can do (capabilities/tools)
4. What it has learned (observations)

### In Everyday Terms

Imagine you're planning a trip:

```
Thought: "I need to get to the airport"
   ↓
Reasoning: "Should I drive or take a train?"
   ↓
Decision: "Train is faster during rush hour"
   ↓
Action: Check train schedule
   ↓
Observation: Next train in 30 minutes
   ↓
Reasoning: "30 minutes is enough time to get to station"
   ↓
Decision: Book train ticket
```

This is **stepwise reasoning** — not a single prompt, but a chain of decisions.

---

## 1.2 How Reasoning Differs from Text Generation

| Aspect | Text Generation | Agent Reasoning |
|--------|----------------|------------------|
| **Input** | Prompt + context | State + goal + observations |
| **Process** | Single LLM call | Multiple steps with decisions |
| **Output** | Text response | Decision + action + new state |
| **Feedback** | None (one-shot) | Iterative (observe → decide → act) |
| **Memory** | Context window only | Persistent state across steps |
| **Debuggability** | Hard (black box) | Easy (trace each step) |

### Example: Text Generation

```
Prompt: "Summarize this article about transformers"
   ↓
[LLM generates summary]
   ↓
Done.
```

**One step. No reasoning trace. No intermediate decisions.**

### Example: Agent Reasoning

```
Goal: "Summarize recent research on transformers"
   ↓
Reasoning: "I need to find recent papers"
   ↓
Action: Search academic database
   ↓
Observation: Found 10 papers
   ↓
Reasoning: "Which papers are most relevant?"
   ↓
Action: Rank by citation count and date
   ↓
Observation: Top 3 papers identified
   ↓
Reasoning: "Should I read abstracts or full text?"
   ↓
Action: Read abstracts
   ↓
Observation: Key findings extracted
   ↓
Reasoning: "Now I can synthesize"
   ↓
Action: Generate summary
   ↓
Done.
```

**Multiple steps. Each step has explicit reasoning. Decisions are observable.**

---

## 1.3 Why Reasoning Must Be Externalized

LLMs can "think" internally (chain-of-thought), but for agents, **internal reasoning is insufficient**.

### The Problem with Internal Reasoning

```python
# What the LLM sees:
prompt = """
You are a helpful agent. Think step by step.

Task: Find papers on transformers and summarize.
"""

# LLM internal monologue (hidden):
# "First I'll search... then I'll read... then I'll summarize..."
# ↓
# [Generates text that LOOKS like it did those steps]
# ↓
response = "Based on recent papers, transformers use attention..."
```

**Problems:**
1. ❌ Did the agent actually search? **Unknown.**
2. ❌ Which papers did it find? **Unknown.**
3. ❌ Why did it choose those papers? **Unknown.**
4. ❌ Can we verify the summary? **No.**
5. ❌ Can we debug if it's wrong? **No.**

### The Solution: Externalized Reasoning

```python
# Agent explicitly records its reasoning:
state = {
    "thought": "I need to search for papers",
    "action": "search_database",
    "action_input": {"query": "transformers 2024"},
    "observation": "Found 10 papers",
    "next_thought": "I should rank by relevance",
    # ... and so on
}
```

**Benefits:**
1. ✅ Every decision is recorded
2. ✅ Every action is traceable
3. ✅ Observations are logged
4. ✅ We can debug each step
5. ✅ We can validate correctness

---

## 1.4 Implicit vs Explicit Reasoning

### Implicit Reasoning (Hidden)

```
┌─────────────────────────────┐
│  Prompt with instructions   │
└──────────────┬──────────────┘
               │
               ▼
       ┌──────────────┐
       │     LLM      │  ← Reasoning happens here (invisible)
       └──────┬───────┘
              │
              ▼
    ┌──────────────────┐
    │  Text Response   │
    └──────────────────┘
```

**Characteristics:**
- Reasoning is internal to the LLM
- No intermediate steps visible
- Cannot inspect or validate
- Looks like magic

### Explicit Reasoning (Externalized)

```
┌─────────────┐
│  Goal/Task  │
└──────┬──────┘
       │
       ▼
  ┌─────────────────┐
  │  Thought: ...   │  ← Recorded in state
  └────────┬────────┘
           │
           ▼
  ┌─────────────────┐
  │  Action: ...    │  ← Recorded in state
  └────────┬────────┘
           │
           ▼
  ┌─────────────────┐
  │ Observation: ...│  ← Recorded in state
  └────────┬────────┘
           │
           ▼
     [Repeat until done]
```

**Characteristics:**
- Reasoning is externalized to state
- Each step is observable
- Can inspect and validate
- Transparent and debuggable

---

## 1.5 Key Insight: Reasoning ≠ Chain-of-Thought

Many tutorials show "chain-of-thought" prompting:

```
Prompt: "Think step-by-step: What is 15% of 80?"
Response: "Let me think:
1. 10% of 80 is 8
2. 5% is half of that, so 4
3. 15% = 10% + 5% = 8 + 4 = 12"
```

This is **internal reasoning made visible**, but:
- It's still in the LLM's response (not structured)
- Cannot be validated programmatically
- Cannot branch based on intermediate results
- Cannot call tools between steps

**Agent reasoning is different:**
- Structured (fields in state)
- Actionable (can call tools)
- Iterative (observe → reason → act)
- Observable (every step is logged)

---

## 🎯 Key Takeaway: Section 1

> **Agent reasoning is the externalized, structured, iterative process by which an agent decides what to do next. Unlike text generation or chain-of-thought, agent reasoning produces observable decision traces that can be validated, debugged, and audited.**

---

<a id="section-2"></a>
# Section 2: Stateless Reasoning vs Structured Reasoning

Let's explore the spectrum from simple prompting to structured reasoning.

---

## 2.1 One-Shot Prompting (No Reasoning)

The simplest form: ask a question, get an answer.

### Conceptual Flow

```
User Question
     ↓
  [LLM]
     ↓
  Answer
```

### Characteristics

- **No intermediate steps**: Question → Answer directly
- **No memory**: Each question is independent
- **No reasoning trace**: We don't know how the LLM arrived at the answer
- **No tool use**: Cannot search, calculate, or take actions

### When This Works

✅ Simple factual questions  
✅ Well-scoped tasks within context  
✅ No need for verification  

### When This Fails

❌ Multi-step problems  
❌ Needs external information  
❌ Requires verification  
❌ Must be auditable  

---

## 2.2 Hidden Chain-of-Thought

We prompt the LLM to "think step-by-step" but the reasoning stays in the response text.

### Conceptual Flow

```
User Question + "Think step-by-step"
     ↓
  [LLM]
     ↓
  Response with reasoning text:
  "Step 1: ...
   Step 2: ...
   Therefore: ..."
```

### Characteristics

- **Reasoning is visible**: We can read the steps
- **But it's unstructured**: Reasoning is in natural language text
- **Cannot branch**: Can't take different actions based on intermediate steps
- **Cannot validate**: Can't programmatically check if reasoning is sound
- **No tool use**: Still just text generation

### Example

```
Prompt: "Calculate the total cost of 3 items at $15.99 each with 8% tax. Think step-by-step."

Response:
"Let me work through this:
Step 1: Cost of 3 items = 3 × $15.99 = $47.97
Step 2: Tax = $47.97 × 0.08 = $3.84
Step 3: Total = $47.97 + $3.84 = $51.81

The total cost is $51.81."
```

### The Problem

This looks like reasoning, but:
1. The math might be wrong (we can't verify programmatically)
2. We can't extract intermediate values for further processing
3. We can't branch (e.g., "if tax > $5, apply discount")
4. The reasoning exists only in text, not in state

---

## 2.3 Why Hidden Reasoning Breaks Debuggability

### Scenario: Research Agent

Imagine an agent that should:
1. Search for papers
2. Filter by relevance
3. Summarize findings

#### Hidden Reasoning Approach

```
Prompt: """
You are a research agent. Find recent papers on transformers and summarize.
Think step-by-step.
"""

Response:
"Step 1: I would search for papers on transformers from 2024
Step 2: I would filter for high-impact papers
Step 3: Based on my knowledge, here's a summary:
Transformers have evolved to..."
```

**Problems:**

| Question | Answer with Hidden Reasoning |
|----------|------------------------------|
| Did the agent actually search? | Unknown (it claims to, but didn't) |
| Which papers were found? | None (it hallucinated) |
| How were they filtered? | Unknown (no actual filtering occurred) |
| Is the summary accurate? | Cannot verify (no source papers) |
| Can we debug if wrong? | No (no intermediate data) |

---

## 2.4 Why Reasoning Artifacts Belong in State, Not Prompts

### What Are Reasoning Artifacts?

Reasoning artifacts are the **outputs of reasoning steps**:

- The thought/decision made
- The action chosen
- The observation received
- Intermediate conclusions
- Why a particular path was taken

### Why They Belong in State

| If in Prompts (Bad) | If in State (Good) |
|---------------------|--------------------|
| Invisible to code | Accessible to code |
| Cannot be validated | Can be validated |
| Cannot be logged separately | Can be logged separately |
| Hard to debug | Easy to debug |
| Lost after response | Preserved across steps |
| Unstructured text | Structured data |

### Example: State-Based Reasoning

```python
# Conceptual structure (no executable code yet)
reasoning_state = {
    "step": 1,
    "thought": "I need to search for papers",
    "action": "search",
    "action_input": {"query": "transformers 2024"},
    "observation": "Found 10 papers: [...]",
    "reasoning_trace": [
        "Step 1: Search identified",
        "Step 2: Papers retrieved",
        # ...
    ]
}
```

Now:
- ✅ We know exactly what action was taken
- ✅ We have the actual search results
- ✅ We can validate the observation
- ✅ We can debug if something goes wrong
- ✅ The reasoning is structured and queryable

---

## 2.5 The Spectrum of Reasoning Approaches

```
LEAST STRUCTURED ────────────────────────► MOST STRUCTURED

One-Shot          Chain of         Hidden ReAct        Explicit
Prompting         Thought          (in text)          Reasoning
                                                      (in state)
   │                 │                 │                  │
   │                 │                 │                  │
   ▼                 ▼                 ▼                  ▼

No trace       Text trace      Text-based         Structured
No steps       Visible steps   steps              state fields
Not           Not              Not fully          Fully
debuggable    validatable      externalized       observable
```

**We want to be on the right side of this spectrum for production agents.**

---

## 🎯 Key Takeaway: Section 2

> **Hidden reasoning (chain-of-thought in prompts) is insufficient for agents. Reasoning must be externalized into structured state where each thought, action, and observation is a first-class field that can be validated, logged, and debugged.**

---

<a id="section-3"></a>
# Section 3: Reasoning as a Process, Not a Prompt

The fundamental shift: reasoning is not what you write in a prompt—it's the **process** the agent executes.

---

## 3.1 Reasoning as Stepwise Decision-Making

True agent reasoning is a **loop**, not a single call.

### The Basic Loop

```
1. Look at current state
   ↓
2. Decide what to do next (reasoning)
   ↓
3. Take an action
   ↓
4. Observe the result
   ↓
5. Update state
   ↓
[Repeat until goal achieved]
```

### Concrete Example

**Goal**: Book a flight to Paris

```
State: {goal: "Book flight to Paris", current_step: null}
   ↓
Reasoning: "I need to know the dates first"
   ↓
Action: Ask user for dates
   ↓
Observation: User says "Dec 15-22"
   ↓
State Update: {dates: "Dec 15-22", current_step: "dates_obtained"}
   ↓
Reasoning: "Now I should search for flights"
   ↓
Action: Search flights
   ↓
Observation: Found 5 flights
   ↓
State Update: {flights: [...], current_step: "flights_found"}
   ↓
Reasoning: "I should present options to user"
   ↓
...
```

Notice:
- **Each reasoning step** looks at the current state
- **Each action** modifies the world (or gathers information)
- **Each observation** provides new information
- **State grows** with each iteration

---

## 3.2 Thought → Action → Observation (The ReAct Pattern)

This pattern is called **ReAct** (Reasoning + Acting).

### The Core Loop

```
┌─────────────────────────────────────┐
│                                     │
│   ┌───────────┐                    │
│   │  THOUGHT  │  "What should I do?" │
│   └─────┬─────┘                    │
│         │                           │
│         ▼                           │
│   ┌───────────┐                    │
│   │  ACTION   │  "Do this"          │
│   └─────┬─────┘                    │
│         │                           │
│         ▼                           │
│   ┌────────────┐                   │
│   │OBSERVATION │  "I learned..."    │
│   └─────┬──────┘                   │
│         │                           │
│         └────────────┐              │
│                      │              │
└──────────────────────┼──────────────┘
                       │
                  [Repeat]
```

### What Each Component Means

#### THOUGHT (Reasoning)
- "What do I know?"
- "What do I need to know?"
- "What action should I take?"
- "Am I done?"

**Stored in state as:** `current_thought` or `reasoning`

#### ACTION (Execution)
- Call a tool (search, calculator, API)
- Query a database
- Ask the user a question
- Generate text

**Stored in state as:** `action` and `action_input`

#### OBSERVATION (Result)
- The result of the action
- New information learned
- Errors or failures

**Stored in state as:** `observation`

---

## 3.3 Why Agents Reason Between Steps, Not Inside Prompts

### The Key Difference

| Inside a Prompt | Between Steps |
|-----------------|---------------|
| "Think about X, then Y, then Z" | Step 1: Think → Act → Observe<br>Step 2: Think → Act → Observe |
| LLM generates all at once | Agent executes iteratively |
| Cannot adapt mid-reasoning | Can adapt based on observations |
| No real-world feedback | Real-world feedback after each action |
| Hallucination risk high | Grounded in actual observations |

### Example: Why Between-Steps Matters

#### Approach 1: Inside Prompt (Bad)

```
Prompt: """
Search for papers on transformers, 
then filter for the top 3, 
then summarize them.
"""

Result:
"I found papers A, B, C. 
The top 3 are A, B, C.
Summary: ..."
```

**Problem**: The LLM didn't actually search. It hallucinated.

#### Approach 2: Between Steps (Good)

```
Step 1:
  Thought: "I should search"
  Action: search("transformers")
  Observation: [actual search results]

Step 2:
  Thought: "I got 10 papers, I should filter"
  Action: filter_by_citations(top=3)
  Observation: [3 specific papers]

Step 3:
  Thought: "Now I can summarize these specific papers"
  Action: summarize([papers])
  Observation: [actual summary based on real papers]
```

**Why This Works**:
- ✅ Actual search happened
- ✅ Actual papers were retrieved
- ✅ Summary is based on real data
- ✅ Each step is verifiable

---

## 3.4 Reasoning Design Checklist

Use this checklist when designing agent reasoning:

### ☑️ Reasoning Design Checklist

| # | Question | Check |
|---|----------|-------|
| 1 | Is reasoning **externalized** (not hidden in prompts)? | ☐ |
| 2 | Is each thought **recorded in state**? | ☐ |
| 3 | Is each action **explicitly defined**? | ☐ |
| 4 | Is each observation **stored for future steps**? | ☐ |
| 5 | Can I **trace the reasoning path** from start to finish? | ☐ |
| 6 | Can I **validate** that actions were actually taken? | ☐ |
| 7 | Can I **debug** why a decision was made? | ☐ |
| 8 | Does reasoning **adapt** based on observations? | ☐ |
| 9 | Is the reasoning loop **iterative** (not one-shot)? | ☐ |
| 10 | Can I **replay** the reasoning for audit purposes? | ☐ |

If you answered "yes" to all 10, your reasoning design is solid.

---

## 3.5 Conceptual Reasoning Flow (Non-Code)

Here's a template for designing reasoning flows before writing code:

```
AGENT REASONING FLOW TEMPLATE
═══════════════════════════════

GOAL: [What the agent is trying to achieve]

INITIAL STATE:
  • What we know: [Initial context]
  • What we need: [Information gaps]

REASONING LOOP:

  ITERATION 1:
    Thought: [What should I do first?]
    Action: [What action will I take?]
    Expected Observation: [What might I learn?]
    State Update: [What changes in state?]

  ITERATION 2:
    Thought: [Given observation, what next?]
    Action: [Next action]
    Expected Observation: [What might I learn?]
    State Update: [What changes?]

  ITERATION 3:
    ...

  TERMINATION CONDITION:
    When: [What indicates we're done?]
    Final Output: [What do we return?]

VALIDATION CHECKS:
  • Can we verify each action was taken?
  • Can we trace the reasoning path?
  • Can we reproduce the result?
```

### Example: Research Agent

```
GOAL: Summarize recent papers on transformers

INITIAL STATE:
  • What we know: Topic is "transformers"
  • What we need: Actual recent papers

REASONING LOOP:

  ITERATION 1:
    Thought: "I need to search for papers"
    Action: search_papers(query="transformers", year=2024)
    Expected Observation: List of papers with metadata
    State Update: papers_found = [list]

  ITERATION 2:
    Thought: "I have 20 papers, too many. I should filter."
    Action: filter_by_citations(papers, top=5)
    Expected Observation: Top 5 papers by citation
    State Update: selected_papers = [top 5]

  ITERATION 3:
    Thought: "Now I can read abstracts and summarize"
    Action: extract_abstracts(selected_papers)
    Expected Observation: Abstract text for each paper
    State Update: abstracts = [texts]

  ITERATION 4:
    Thought: "I have enough information to synthesize"
    Action: generate_summary(abstracts)
    Expected Observation: Summary text
    State Update: final_summary = text

  TERMINATION CONDITION:
    When: final_summary is not None
    Final Output: Return final_summary

VALIDATION CHECKS:
  ✓ Verify papers_found is not empty
  ✓ Verify selected_papers has 5 or fewer
  ✓ Verify abstracts were actually extracted
  ✓ Verify summary references actual papers
```

---

## 3.6 The Mental Shift

### Old Mindset (Prompting)

```
"I need to write the perfect prompt that
 tells the LLM exactly what to do."
```

### New Mindset (Reasoning)

```
"I need to design a reasoning loop where
 the agent observes, thinks, acts, and
 adapts based on real feedback."
```

This shift is fundamental to building production agents.

---

## 🎯 Key Takeaway: Section 3

> **Agent reasoning is a process, not a prompt. It's an iterative loop of Thought → Action → Observation where each step is externalized into state. Design your reasoning flow before writing code, and ensure every decision is traceable and debuggable.**

---

---

# Summary: Sections 1-3

| Section | Key Concept |
|---------|-------------|
| **Section 1** | Reasoning is externalized decision-making, not internal chain-of-thought |
| **Section 2** | Hidden reasoning breaks debuggability; artifacts must live in state |
| **Section 3** | Reasoning is an iterative process (Thought → Action → Observation) |

---

## Next Sections Preview

In upcoming sections, we will:

- **Section 4:** Implement explicit reasoning patterns in code
- **Section 5:** Store reasoning artifacts in state
- **Section 6:** Build a complete ReAct agent
- **Section 7:** Visualize reasoning flows
- **Section 8:** Multi-agent reasoning coordination
- **Section 9:** Common reasoning mistakes
- **Section 10:** Final reasoning framework

---

<a id="section-4"></a>
# Section 4: Single-Agent Step-by-Step Reasoning

Now let's implement a real agent with **explicit reasoning** stored in state.

---

## 4.1 Setup: Imports and Helper Functions

First, we'll set up our imports and create helper functions for displaying reasoning state.

In [ ]:
# Required imports
from typing import TypedDict, Optional, List, Dict, Any
from dataclasses import dataclass, field, asdict
from datetime import datetime
from enum import Enum
from copy import deepcopy
import json
from pprint import pprint

# Helper function to print section headers
def print_section_header(title: str, width: int = 60):
    """Print a formatted section header"""
    print("\n" + "=" * width)
    print(f"  {title}")
    print("=" * width + "\n")

# Helper function to display reasoning state
def display_reasoning_state(state: Dict[str, Any], step_number: int = None):
    """Display the current reasoning state in a readable format"""
    if step_number is not None:
        print(f"\n{'─' * 60}")
        print(f"  REASONING STEP {step_number}")
        print('─' * 60)
    
    for key, value in state.items():
        if isinstance(value, list) and len(value) > 3:
            print(f"  {key}: [{len(value)} items]")
        elif isinstance(value, str) and len(value) > 100:
            print(f"  {key}: {value[:100]}...")
        else:
            print(f"  {key}: {value}")
    print()

print("✓ Imports and helper functions loaded")

## 4.2 Reasoning State Schema

We'll define a structured state that explicitly captures all reasoning artifacts.

In [ ]:
@dataclass
class ReasoningStep:
    """A single reasoning step with thought, action, and observation"""
    step_number: int
    thought: str
    action: str
    action_input: Dict[str, Any]
    observation: Optional[str] = None
    timestamp: str = field(default_factory=lambda: datetime.now().isoformat())

@dataclass
class ResearchAgentState:
    """State for a research agent with explicit reasoning"""
    goal: str
    current_step: int = 0
    reasoning_trace: List[ReasoningStep] = field(default_factory=list)
    papers_found: List[Dict[str, str]] = field(default_factory=list)
    selected_papers: List[Dict[str, str]] = field(default_factory=list)
    final_summary: Optional[str] = None
    is_complete: bool = False
    
    def add_reasoning_step(self, thought: str, action: str, action_input: Dict[str, Any], observation: Optional[str] = None):
        """Add a new reasoning step to the trace"""
        self.current_step += 1
        step = ReasoningStep(
            step_number=self.current_step,
            thought=thought,
            action=action,
            action_input=action_input,
            observation=observation
        )
        self.reasoning_trace.append(step)
        return step
    
    def get_latest_step(self) -> Optional[ReasoningStep]:
        """Get the most recent reasoning step"""
        return self.reasoning_trace[-1] if self.reasoning_trace else None

# Create an example state
example_state = ResearchAgentState(goal="Summarize recent papers on transformers")
print("✓ ResearchAgentState schema defined")
print("\nInitial state:")
print(f"  Goal: {example_state.goal}")
print(f"  Current step: {example_state.current_step}")
print(f"  Reasoning trace: {len(example_state.reasoning_trace)} steps")

## 4.3 Simulated Tools (Mock Functions)

Since we don't have real APIs, we'll simulate search and retrieval functions.

In [ ]:
# Simulated database of papers
MOCK_PAPER_DATABASE = [
    {"title": "Attention Is All You Need", "year": 2017, "citations": 95000, "abstract": "We propose the Transformer..."},
    {"title": "BERT: Pre-training of Deep Bidirectional Transformers", "year": 2018, "citations": 75000, "abstract": "We introduce BERT..."},
    {"title": "GPT-3: Language Models are Few-Shot Learners", "year": 2020, "citations": 50000, "abstract": "We show that scaling up language models..."},
    {"title": "Vision Transformer", "year": 2021, "citations": 30000, "abstract": "We apply transformers to image recognition..."},
    {"title": "Flamingo: Visual Language Models", "year": 2022, "citations": 15000, "abstract": "We present Flamingo..."},
    {"title": "LLaMA: Open Foundation Models", "year": 2023, "citations": 12000, "abstract": "We release LLaMA..."},
    {"title": "Mixtral: Mixture of Experts", "year": 2024, "citations": 5000, "abstract": "We present Mixtral..."},
    {"title": "Transformers for Time Series", "year": 2024, "citations": 3000, "abstract": "We adapt transformers for time series..."},
]

def search_papers(query: str, year_from: int = None) -> List[Dict[str, Any]]:
    """Simulate searching for papers"""
    results = MOCK_PAPER_DATABASE.copy()
    
    # Filter by year if specified
    if year_from:
        results = [p for p in results if p["year"] >= year_from]
    
    # Simple keyword matching (just for demo)
    query_lower = query.lower()
    if "vision" in query_lower or "image" in query_lower:
        results = [p for p in results if "vision" in p["title"].lower() or "image" in p["abstract"].lower()]
    
    return results

def filter_papers_by_citations(papers: List[Dict[str, Any]], top_n: int = 3) -> List[Dict[str, Any]]:
    """Filter papers by citation count"""
    sorted_papers = sorted(papers, key=lambda x: x["citations"], reverse=True)
    return sorted_papers[:top_n]

def extract_abstracts(papers: List[Dict[str, Any]]) -> List[str]:
    """Extract abstracts from papers"""
    return [p["abstract"] for p in papers]

def synthesize_summary(abstracts: List[str], papers: List[Dict[str, Any]]) -> str:
    """Generate a summary from abstracts (simulated)"""
    titles = [p["title"] for p in papers]
    summary = f"Analysis of {len(papers)} papers: {', '.join(titles)}.\n"
    summary += f"Key findings: These papers demonstrate the evolution of transformer architectures "
    summary += f"from the original attention mechanism to modern applications."
    return summary

print("✓ Mock tools defined (search_papers, filter_papers_by_citations, extract_abstracts, synthesize_summary)")

## 4.4 Implementing the Research Agent

Now we'll implement an agent that **explicitly reasons** through each step.

In [ ]:
def research_agent_step_1_search(state: ResearchAgentState) -> ResearchAgentState:
    """Step 1: Search for papers"""
    # THOUGHT: What should I do?
    thought = "I need to search for recent papers on transformers"
    
    # ACTION: What will I do?
    action = "search_papers"
    action_input = {"query": "transformers", "year_from": 2020}
    
    # Execute the action
    papers = search_papers(**action_input)
    
    # OBSERVATION: What did I learn?
    observation = f"Found {len(papers)} papers from 2020 onwards"
    
    # Record the reasoning step
    state.add_reasoning_step(thought, action, action_input, observation)
    
    # Update state with results
    state.papers_found = papers
    
    return state

def research_agent_step_2_filter(state: ResearchAgentState) -> ResearchAgentState:
    """Step 2: Filter papers by relevance"""
    # THOUGHT: Given what I know, what next?
    thought = f"I found {len(state.papers_found)} papers. I should filter to the most cited ones."
    
    # ACTION: What will I do?
    action = "filter_papers_by_citations"
    action_input = {"papers": "papers_found", "top_n": 3}
    
    # Execute the action
    selected = filter_papers_by_citations(state.papers_found, top_n=3)
    
    # OBSERVATION: What did I learn?
    observation = f"Selected top 3 papers by citations: {[p['title'] for p in selected]}"
    
    # Record the reasoning step
    state.add_reasoning_step(thought, action, action_input, observation)
    
    # Update state with results
    state.selected_papers = selected
    
    return state

def research_agent_step_3_extract(state: ResearchAgentState) -> ResearchAgentState:
    """Step 3: Extract abstracts"""
    # THOUGHT: What's next?
    thought = "Now I need to read the abstracts of the selected papers"
    
    # ACTION: What will I do?
    action = "extract_abstracts"
    action_input = {"papers": "selected_papers"}
    
    # Execute the action
    abstracts = extract_abstracts(state.selected_papers)
    
    # OBSERVATION: What did I learn?
    observation = f"Extracted {len(abstracts)} abstracts"
    
    # Record the reasoning step
    state.add_reasoning_step(thought, action, action_input, observation)
    
    return state

def research_agent_step_4_synthesize(state: ResearchAgentState) -> ResearchAgentState:
    """Step 4: Synthesize summary"""
    # THOUGHT: Am I ready to conclude?
    thought = "I have the abstracts. Now I can synthesize a summary."
    
    # ACTION: What will I do?
    action = "synthesize_summary"
    action_input = {"abstracts": "extracted", "papers": "selected_papers"}
    
    # Execute the action
    abstracts = extract_abstracts(state.selected_papers)
    summary = synthesize_summary(abstracts, state.selected_papers)
    
    # OBSERVATION: What's the result?
    observation = "Summary generated successfully"
    
    # Record the reasoning step
    state.add_reasoning_step(thought, action, action_input, observation)
    
    # Update state with final result
    state.final_summary = summary
    state.is_complete = True
    
    return state

print("✓ Research agent steps defined (step_1_search, step_2_filter, step_3_extract, step_4_synthesize)")

## 4.5 Running the Agent and Observing Reasoning

Let's execute each step and observe the reasoning evolution.

In [ ]:
print_section_header("RESEARCH AGENT EXECUTION")

# Initialize state
state = ResearchAgentState(goal="Summarize recent transformer papers")

print(f"Initial Goal: {state.goal}\n")

# Execute Step 1
print("▶ Executing Step 1: Search")
state = research_agent_step_1_search(state)
latest_step = state.get_latest_step()
print(f"  Thought: {latest_step.thought}")
print(f"  Action: {latest_step.action}({latest_step.action_input})")
print(f"  Observation: {latest_step.observation}")
print(f"  Papers found: {len(state.papers_found)}")

# Execute Step 2
print("\n▶ Executing Step 2: Filter")
state = research_agent_step_2_filter(state)
latest_step = state.get_latest_step()
print(f"  Thought: {latest_step.thought}")
print(f"  Action: {latest_step.action}({latest_step.action_input})")
print(f"  Observation: {latest_step.observation}")
print(f"  Selected papers: {len(state.selected_papers)}")

# Execute Step 3
print("\n▶ Executing Step 3: Extract Abstracts")
state = research_agent_step_3_extract(state)
latest_step = state.get_latest_step()
print(f"  Thought: {latest_step.thought}")
print(f"  Action: {latest_step.action}({latest_step.action_input})")
print(f"  Observation: {latest_step.observation}")

# Execute Step 4
print("\n▶ Executing Step 4: Synthesize Summary")
state = research_agent_step_4_synthesize(state)
latest_step = state.get_latest_step()
print(f"  Thought: {latest_step.thought}")
print(f"  Action: {latest_step.action}({latest_step.action_input})")
print(f"  Observation: {latest_step.observation}")

print(f"\n{'=' * 60}")
print("FINAL RESULT")
print('=' * 60)
print(f"Complete: {state.is_complete}")
print(f"Summary: {state.final_summary}")
print(f"\nTotal reasoning steps: {len(state.reasoning_trace)}")

## 4.6 Inspecting the Complete Reasoning Trace

Let's examine the full reasoning trace to see every decision the agent made.

In [ ]:
print_section_header("COMPLETE REASONING TRACE")

for step in state.reasoning_trace:
    print(f"Step {step.step_number}:")
    print(f"  💭 Thought: {step.thought}")
    print(f"  🔧 Action: {step.action}")
    print(f"  📥 Input: {step.action_input}")
    print(f"  👁️  Observation: {step.observation}")
    print(f"  🕐 Timestamp: {step.timestamp}")
    print()

print("=" * 60)
print("WHY THIS MATTERS")
print("=" * 60)
print("""
✅ Every decision is recorded
✅ Every action is traceable
✅ Every observation is logged
✅ We can debug each step
✅ We can validate correctness
✅ We can audit the reasoning process

This is the difference between:
  ❌ "The agent said it searched" (unverifiable)
  ✅ "The agent searched and found 5 papers" (verified in state)
""")

## 🎯 Key Takeaway: Section 4

> **By externalizing reasoning into structured state, we transform agent behavior from a black box into a transparent, debuggable, auditable process. Every thought, action, and observation becomes a first-class artifact that can be inspected, validated, and replayed.**

---

<a id="section-5"></a>
# Section 5: Reasoning Validation & Evaluation

Having reasoning in state is not enough—we must **validate** that the reasoning is correct.

---

## 5.1 Why Validation Matters

Without validation, agents can:
- Skip critical reasoning steps
- Hallucinate conclusions without evidence
- Claim to have done work they never did
- Produce unreliable results

**Validation ensures the reasoning trace reflects actual execution.**

---

## 5.2 Validation Schema

In [ ]:
class ReasoningValidationError(Exception):
    """Raised when reasoning validation fails"""
    pass

def validate_reasoning_sequence(state: ResearchAgentState, expected_actions: List[str]) -> bool:
    """Validate that reasoning steps occurred in the expected order"""
    actual_actions = [step.action for step in state.reasoning_trace]
    
    if len(actual_actions) != len(expected_actions):
        raise ReasoningValidationError(
            f"Expected {len(expected_actions)} steps, but found {len(actual_actions)}"
        )
    
    for i, (actual, expected) in enumerate(zip(actual_actions, expected_actions)):
        if actual != expected:
            raise ReasoningValidationError(
                f"Step {i+1}: Expected action '{expected}', but found '{actual}'"
            )
    
    print(f"✓ Reasoning sequence validated: {len(expected_actions)} steps in correct order")
    return True

def validate_no_skipped_steps(state: ResearchAgentState) -> bool:
    """Ensure no steps were skipped"""
    for i, step in enumerate(state.reasoning_trace, 1):
        if step.step_number != i:
            raise ReasoningValidationError(
                f"Step numbering error: Expected step {i}, found step {step.step_number}"
            )
    
    print(f"✓ No skipped steps: All {len(state.reasoning_trace)} steps present")
    return True

def validate_observations_exist(state: ResearchAgentState) -> bool:
    """Ensure all reasoning steps have observations"""
    for step in state.reasoning_trace:
        if step.observation is None or step.observation == "":
            raise ReasoningValidationError(
                f"Step {step.step_number} has no observation (possible hallucination)"
            )
    
    print(f"✓ All {len(state.reasoning_trace)} steps have observations")
    return True

def validate_evidence_before_conclusion(state: ResearchAgentState) -> bool:
    """Ensure evidence was gathered before final conclusion"""
    if state.is_complete and state.final_summary:
        # Check that we have selected papers
        if not state.selected_papers:
            raise ReasoningValidationError(
                "Final summary generated without selecting any papers (hallucination)"
            )
        
        # Check that we actually searched
        search_steps = [s for s in state.reasoning_trace if s.action == "search_papers"]
        if not search_steps:
            raise ReasoningValidationError(
                "Final summary generated without searching for papers"
            )
        
        print("✓ Evidence validated: Summary based on actual retrieved papers")
    
    return True

print("✓ Validation functions defined")

## 5.3 Validating Our Research Agent

Let's validate the reasoning trace from Section 4.

In [ ]:
print_section_header("VALIDATING RESEARCH AGENT REASONING")

# Define expected sequence
expected_sequence = [
    "search_papers",
    "filter_papers_by_citations", 
    "extract_abstracts",
    "synthesize_summary"
]

# Run all validations
try:
    validate_reasoning_sequence(state, expected_sequence)
    validate_no_skipped_steps(state)
    validate_observations_exist(state)
    validate_evidence_before_conclusion(state)
    
    print("\n" + "=" * 60)
    print("✅ ALL VALIDATIONS PASSED")
    print("=" * 60)
    print("The reasoning trace is valid and trustworthy.")
    
except ReasoningValidationError as e:
    print(f"\n❌ VALIDATION FAILED: {e}")

## 5.4 Failure Case: Skipped Reasoning

Let's demonstrate what happens when an agent skips a critical step.

In [ ]:
print_section_header("FAILURE CASE: Skipped Search Step")

# Create a broken agent that skips the search step
broken_state = ResearchAgentState(goal="Summarize papers")

# Oops! Skip search and go straight to filtering (WRONG!)
broken_state.add_reasoning_step(
    thought="I'll just filter papers",
    action="filter_papers_by_citations",
    action_input={"papers": [], "top_n": 3},
    observation="No papers to filter"
)

# Try to synthesize without data
broken_state.add_reasoning_step(
    thought="I'll synthesize a summary",
    action="synthesize_summary",
    action_input={"abstracts": [], "papers": []},
    observation="Summary generated"
)

# Hallucinate a summary
broken_state.final_summary = "Based on recent papers, transformers are very popular..."
broken_state.is_complete = True

print("Broken agent created (skipped search step)")
print(f"Reasoning steps: {len(broken_state.reasoning_trace)}")
print(f"Papers found: {len(broken_state.papers_found)}")
print(f"Summary: {broken_state.final_summary}")

# Now try to validate
print("\n" + "-" * 60)
print("Attempting validation...")
print("-" * 60)

try:
    validate_evidence_before_conclusion(broken_state)
    print("✅ Validation passed (this should NOT happen)")
except ReasoningValidationError as e:
    print(f"❌ Validation caught the error!")
    print(f"   Error: {e}")
    print("\n✓ This is GOOD - validation detected the hallucination")

## 5.5 Corrected Reasoning Trace

Now let's show the corrected version with proper reasoning.

In [ ]:
print_section_header("CORRECTED: Proper Reasoning Trace")

# Create a corrected agent
corrected_state = ResearchAgentState(goal="Summarize papers")

# Step 1: Actually search
corrected_state = research_agent_step_1_search(corrected_state)
print(f"✓ Step 1: Searched and found {len(corrected_state.papers_found)} papers")

# Step 2: Filter
corrected_state = research_agent_step_2_filter(corrected_state)
print(f"✓ Step 2: Filtered to {len(corrected_state.selected_papers)} papers")

# Step 3: Extract
corrected_state = research_agent_step_3_extract(corrected_state)
print(f"✓ Step 3: Extracted abstracts")

# Step 4: Synthesize
corrected_state = research_agent_step_4_synthesize(corrected_state)
print(f"✓ Step 4: Generated summary")

# Now validate
print("\n" + "-" * 60)
print("Validating corrected reasoning...")
print("-" * 60)

try:
    validate_evidence_before_conclusion(corrected_state)
    print("\n✅ VALIDATION PASSED")
    print("The corrected agent has proper evidence for its conclusions.")
except ReasoningValidationError as e:
    print(f"❌ Validation failed: {e}")

print("\n" + "=" * 60)
print("COMPARISON")
print("=" * 60)
print(f"Broken agent:    {len(broken_state.papers_found)} papers, {len(broken_state.reasoning_trace)} steps")
print(f"Corrected agent: {len(corrected_state.papers_found)} papers, {len(corrected_state.reasoning_trace)} steps")
print("\nThe corrected agent has actual evidence supporting its conclusions.")

## 🎯 Key Takeaway: Section 5

> **Reasoning without evaluation is unreliable. Validation functions ensure that agents don't skip steps, hallucinate conclusions, or claim work they never did. Always validate that evidence exists before accepting conclusions.**

**Critical principle:** If you can't validate the reasoning, you can't trust the result.

---

<a id="section-6"></a>
# Section 6: Reasoning in a RAG Agent

Now let's apply explicit reasoning to a **Retrieval-Augmented Generation (RAG)** agent.

---

## 6.1 Why RAG Needs Explicit Reasoning

RAG agents must reason about:
1. **Query Analysis**: What information do I need?
2. **Retrieval Decision**: Where should I search?
3. **Evidence Selection**: Which documents are relevant?
4. **Answer Synthesis**: How do I combine evidence into an answer?

Without explicit reasoning, RAG agents can:
- ❌ Cite non-existent documents
- ❌ Generate answers without retrieval
- ❌ Use irrelevant evidence
- ❌ Make unjustified inferences

---

## 6.2 RAG Agent State Schema

In [ ]:
@dataclass
class Document:
    """A retrieved document"""
    id: str
    content: str
    source: str
    relevance_score: float = 0.0

@dataclass
class RAGReasoningStep:
    """Reasoning step specific to RAG"""
    step_number: int
    phase: str  # "query_analysis", "retrieval", "selection", "synthesis"
    reasoning: str
    action: str
    evidence: List[Document] = field(default_factory=list)
    decision: str = ""
    timestamp: str = field(default_factory=lambda: datetime.now().isoformat())

@dataclass
class RAGAgentState:
    """State for a RAG agent with explicit reasoning"""
    query: str
    reasoning_trace: List[RAGReasoningStep] = field(default_factory=list)
    retrieved_documents: List[Document] = field(default_factory=list)
    selected_documents: List[Document] = field(default_factory=list)
    final_answer: Optional[str] = None
    current_step: int = 0
    
    def add_reasoning_step(self, phase: str, reasoning: str, action: str, 
                          evidence: List[Document] = None, decision: str = ""):
        """Add a RAG reasoning step"""
        self.current_step += 1
        step = RAGReasoningStep(
            step_number=self.current_step,
            phase=phase,
            reasoning=reasoning,
            action=action,
            evidence=evidence or [],
            decision=decision
        )
        self.reasoning_trace.append(step)
        return step

print("✓ RAG agent state schema defined")

## 6.3 Mock Document Database and Tools

In [ ]:
# Mock document database
MOCK_DOCUMENTS = [
    Document(id="doc1", content="Transformers use self-attention mechanisms to process sequences in parallel.", 
             source="paper_attention.pdf", relevance_score=0.95),
    Document(id="doc2", content="BERT is a bidirectional encoder trained on masked language modeling.", 
             source="paper_bert.pdf", relevance_score=0.88),
    Document(id="doc3", content="GPT models are autoregressive transformers trained on next-token prediction.", 
             source="paper_gpt.pdf", relevance_score=0.92),
    Document(id="doc4", content="Vision transformers apply the transformer architecture to image patches.", 
             source="paper_vit.pdf", relevance_score=0.75),
    Document(id="doc5", content="The weather today is sunny with a chance of rain.", 
             source="weather.txt", relevance_score=0.05),  # Irrelevant document
]

def retrieve_documents(query: str, top_k: int = 5) -> List[Document]:
    """Simulate document retrieval"""
    # Simple keyword matching for demo
    query_lower = query.lower()
    
    # Score documents based on keyword overlap
    scored_docs = []
    for doc in MOCK_DOCUMENTS:
        score = doc.relevance_score
        # Boost score if query keywords appear
        if any(word in doc.content.lower() for word in query_lower.split()):
            score *= 1.2
        
        scored_docs.append((doc, score))
    
    # Sort by score and return top_k
    scored_docs.sort(key=lambda x: x[1], reverse=True)
    return [doc for doc, score in scored_docs[:top_k]]

def select_relevant_documents(documents: List[Document], threshold: float = 0.7) -> List[Document]:
    """Filter documents by relevance threshold"""
    return [doc for doc in documents if doc.relevance_score >= threshold]

def synthesize_answer(query: str, documents: List[Document]) -> str:
    """Generate answer from selected documents"""
    if not documents:
        return "I don't have enough information to answer this question."
    
    # Simple synthesis (in real system, would use LLM)
    doc_contents = " ".join([doc.content for doc in documents])
    sources = ", ".join([doc.source for doc in documents])
    
    answer = f"Based on the retrieved documents: {doc_contents[:200]}...\n\n"
    answer += f"Sources: {sources}"
    return answer

print("✓ Mock RAG tools defined (retrieve_documents, select_relevant_documents, synthesize_answer)")

## 6.4 Implementing the RAG Agent with Explicit Reasoning

In [ ]:
def rag_step_1_query_analysis(state: RAGAgentState) -> RAGAgentState:
    """Step 1: Analyze the query"""
    reasoning = f"The user is asking: '{state.query}'. I need to identify what information to retrieve."
    decision = "Search for documents about transformers and attention mechanisms"
    
    state.add_reasoning_step(
        phase="query_analysis",
        reasoning=reasoning,
        action="analyze_query",
        decision=decision
    )
    
    return state

def rag_step_2_retrieval(state: RAGAgentState) -> RAGAgentState:
    """Step 2: Retrieve relevant documents"""
    reasoning = "I will search the document database for relevant content"
    
    # Actually retrieve documents
    docs = retrieve_documents(state.query, top_k=5)
    
    decision = f"Retrieved {len(docs)} documents from the database"
    
    state.add_reasoning_step(
        phase="retrieval",
        reasoning=reasoning,
        action="retrieve_documents",
        evidence=docs,
        decision=decision
    )
    
    state.retrieved_documents = docs
    return state

def rag_step_3_selection(state: RAGAgentState) -> RAGAgentState:
    """Step 3: Select relevant documents"""
    reasoning = f"I retrieved {len(state.retrieved_documents)} documents. I need to filter for relevance."
    
    # Filter documents
    selected = select_relevant_documents(state.retrieved_documents, threshold=0.7)
    
    decision = f"Selected {len(selected)} documents with relevance >= 0.7"
    
    state.add_reasoning_step(
        phase="selection",
        reasoning=reasoning,
        action="filter_by_relevance",
        evidence=selected,
        decision=decision
    )
    
    state.selected_documents = selected
    return state

def rag_step_4_synthesis(state: RAGAgentState) -> RAGAgentState:
    """Step 4: Synthesize answer from evidence"""
    reasoning = f"I have {len(state.selected_documents)} relevant documents. I can now generate an answer."
    
    # Synthesize answer
    answer = synthesize_answer(state.query, state.selected_documents)
    
    decision = "Generated answer based on retrieved evidence"
    
    state.add_reasoning_step(
        phase="synthesis",
        reasoning=reasoning,
        action="synthesize_answer",
        evidence=state.selected_documents,
        decision=decision
    )
    
    state.final_answer = answer
    return state

print("✓ RAG agent steps defined")

## 6.5 Running the RAG Agent

In [ ]:
print_section_header("RAG AGENT EXECUTION")

# Initialize RAG state
rag_state = RAGAgentState(query="How do transformers work?")

print(f"Query: {rag_state.query}\n")

# Step 1: Query Analysis
print("▶ Step 1: Query Analysis")
rag_state = rag_step_1_query_analysis(rag_state)
step = rag_state.reasoning_trace[-1]
print(f"  Phase: {step.phase}")
print(f"  Reasoning: {step.reasoning}")
print(f"  Decision: {step.decision}")

# Step 2: Retrieval
print("\n▶ Step 2: Document Retrieval")
rag_state = rag_step_2_retrieval(rag_state)
step = rag_state.reasoning_trace[-1]
print(f"  Phase: {step.phase}")
print(f"  Reasoning: {step.reasoning}")
print(f"  Decision: {step.decision}")
print(f"  Retrieved: {len(rag_state.retrieved_documents)} documents")

# Step 3: Selection
print("\n▶ Step 3: Evidence Selection")
rag_state = rag_step_3_selection(rag_state)
step = rag_state.reasoning_trace[-1]
print(f"  Phase: {step.phase}")
print(f"  Reasoning: {step.reasoning}")
print(f"  Decision: {step.decision}")
print(f"  Selected: {len(rag_state.selected_documents)} documents")
for doc in rag_state.selected_documents:
    print(f"    - {doc.source} (relevance: {doc.relevance_score})")

# Step 4: Synthesis
print("\n▶ Step 4: Answer Synthesis")
rag_state = rag_step_4_synthesis(rag_state)
step = rag_state.reasoning_trace[-1]
print(f"  Phase: {step.phase}")
print(f"  Reasoning: {step.reasoning}")
print(f"  Decision: {step.decision}")

print(f"\n{'=' * 60}")
print("FINAL ANSWER")
print('=' * 60)
print(rag_state.final_answer)

## 6.6 Validating RAG Reasoning

Critical validation: ensure evidence exists before synthesis.

In [ ]:
def validate_rag_reasoning(state: RAGAgentState) -> bool:
    """Validate that RAG agent followed proper reasoning"""
    
    # Check 1: Evidence exists before synthesis
    synthesis_steps = [s for s in state.reasoning_trace if s.phase == "synthesis"]
    if synthesis_steps:
        if not state.selected_documents:
            raise ReasoningValidationError(
                "Answer was synthesized without selecting any documents (hallucination)"
            )
        print("✓ Evidence exists before synthesis")
    
    # Check 2: Retrieval happened before selection
    retrieval_steps = [s for s in state.reasoning_trace if s.phase == "retrieval"]
    selection_steps = [s for s in state.reasoning_trace if s.phase == "selection"]
    
    if selection_steps and not retrieval_steps:
        raise ReasoningValidationError(
            "Selection happened without retrieval"
        )
    print("✓ Retrieval occurred before selection")
    
    # Check 3: Reasoning references retrieved content
    if state.final_answer and state.selected_documents:
        # Check that answer isn't empty
        if not state.final_answer or len(state.final_answer) < 10:
            raise ReasoningValidationError(
                "Final answer is too short or empty"
            )
        print("✓ Final answer generated from evidence")
    
    # Check 4: All steps have reasoning recorded
    for step in state.reasoning_trace:
        if not step.reasoning:
            raise ReasoningValidationError(
                f"Step {step.step_number} ({step.phase}) has no reasoning recorded"
            )
    print(f"✓ All {len(state.reasoning_trace)} steps have reasoning recorded")
    
    return True

print_section_header("VALIDATING RAG AGENT")

try:
    validate_rag_reasoning(rag_state)
    print("\n" + "=" * 60)
    print("✅ RAG REASONING VALIDATED")
    print("=" * 60)
    print("The agent:")
    print("  • Actually retrieved documents")
    print("  • Selected relevant evidence")
    print("  • Generated answer based on real evidence")
    print("  • Recorded all reasoning decisions")
except ReasoningValidationError as e:
    print(f"\n❌ VALIDATION FAILED: {e}")

## 6.7 Inspecting RAG Reasoning Trace

Let's examine the complete reasoning trace to verify every decision.

In [ ]:
print_section_header("COMPLETE RAG REASONING TRACE")

for step in rag_state.reasoning_trace:
    print(f"Step {step.step_number}: {step.phase.upper()}")
    print(f"  💭 Reasoning: {step.reasoning}")
    print(f"  🔧 Action: {step.action}")
    print(f"  📊 Evidence: {len(step.evidence)} documents")
    if step.evidence:
        for doc in step.evidence[:2]:  # Show first 2
            print(f"      - {doc.source}: {doc.content[:60]}...")
    print(f"  ✓ Decision: {step.decision}")
    print()

print("=" * 60)
print("KEY INSIGHTS")
print("=" * 60)
print("""
✅ Every RAG decision is traceable:
   • Query analysis: What information do we need?
   • Retrieval: What documents were found?
   • Selection: Which documents are relevant?
   • Synthesis: How was the answer generated?

✅ Evidence is stored in state:
   • We can verify which documents were used
   • We can check relevance scores
   • We can trace answer back to sources

✅ No hallucination possible:
   • Answer MUST reference actual documents
   • Documents MUST exist in state
   • Validation ensures evidence-before-conclusion

This is production-ready RAG reasoning.
""")

## 🎯 Key Takeaway: Section 6

> **RAG agents must externalize every reasoning decision: query analysis, retrieval, evidence selection, and synthesis. By storing documents and reasoning in state, we prevent hallucination and ensure every answer is grounded in verifiable evidence.**

---

# Summary: Sections 4-6

| Section | Key Implementation |
|---------|-------------------|
| **Section 4** | Single-agent reasoning with explicit thought/action/observation in state |
| **Section 5** | Validation framework to detect skipped steps and hallucinations |
| **Section 6** | RAG agent with evidence-based reasoning and validation |

---

**Next sections (7-10) will cover:**
- Visual reasoning flows
- Multi-agent reasoning coordination
- Common reasoning mistakes
- Production reasoning framework

---

<a id="section-7"></a>
# Section 7: Visual Diagrams of Reasoning Flow

Visual diagrams help us understand how reasoning flows through an agent system.

---

## 7.1 The Basic Reasoning Loop

This is the fundamental pattern all agents follow:

In [ ]:
print_section_header("DIAGRAM 1: Basic Reasoning Loop")

diagram_1 = """
┌─────────────────────────────────────────────────┐
│          AGENT REASONING LOOP                   │
└─────────────────────────────────────────────────┘

              ┌──────────────┐
              │  START       │
              │  (Goal Set)  │
              └──────┬───────┘
                     │
                     ▼
         ┌───────────────────────┐
         │   READ CURRENT STATE  │
         │   - What do I know?   │
         │   - What's happened?  │
         └───────────┬───────────┘
                     │
                     ▼
         ┌───────────────────────┐
         │   💭 THOUGHT          │
         │   - What should I do? │
         │   - Why this action?  │
         └───────────┬───────────┘
                     │
                     ▼
         ┌───────────────────────┐
         │   🔧 ACTION           │
         │   - Execute tool      │
         │   - Call API          │
         │   - Generate text     │
         └───────────┬───────────┘
                     │
                     ▼
         ┌───────────────────────┐
         │   👁️ OBSERVATION      │
         │   - What happened?    │
         │   - New information   │
         └───────────┬───────────┘
                     │
                     ▼
         ┌───────────────────────┐
         │   UPDATE STATE        │
         │   - Store observation │
         │   - Record reasoning  │
         └───────────┬───────────┘
                     │
                     ▼
              ┌──────────────┐
              │  Goal Done?  │
              └──────┬───────┘
                     │
            ┌────────┴────────┐
            │                 │
           NO                YES
            │                 │
            │                 ▼
            │         ┌──────────────┐
            │         │  TERMINATE   │
            │         │  Return Result│
            │         └──────────────┘
            │
            └─────────┐
                      │
                      ▼
              [LOOP BACK TO READ STATE]
"""

print(diagram_1)

print("\n" + "=" * 60)
print("KEY INSIGHTS")
print("=" * 60)
print("""
✓ Reasoning is ITERATIVE, not one-shot
✓ Each cycle adds to state
✓ Observations feed into next thought
✓ Loop continues until goal is met
""")

## 7.2 Thought → Action → Observation (ReAct Pattern)

The core reasoning cycle in detail:

In [ ]:
print_section_header("DIAGRAM 2: Thought → Action → Observation")

diagram_2 = """
ReAct PATTERN: Reasoning + Acting

┌─────────────────────────────────────────────────────────────┐
│                       ITERATION 1                           │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│  💭 THOUGHT:                                                │
│     "I need to find information about X"                    │
│     "The best way is to search the database"                │
│     ↓                                                       │
│  🔧 ACTION:                                                 │
│     search_database(query="X")                              │
│     ↓                                                       │
│  👁️ OBSERVATION:                                            │
│     "Found 5 results: [...]"                                │
│     ↓                                                       │
│  📝 STATE UPDATE:                                           │
│     state.search_results = [5 items]                        │
│                                                             │
└─────────────────────────────────────────────────────────────┘
                            │
                            ▼
┌─────────────────────────────────────────────────────────────┐
│                       ITERATION 2                           │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│  💭 THOUGHT:                                                │
│     "I have 5 results, but which is most relevant?"         │
│     "I should rank them by score"                           │
│     ↓                                                       │
│  🔧 ACTION:                                                 │
│     rank_results(results=state.search_results)              │
│     ↓                                                       │
│  👁️ OBSERVATION:                                            │
│     "Top result has score 0.95"                             │
│     ↓                                                       │
│  📝 STATE UPDATE:                                           │
│     state.selected_result = top_result                      │
│                                                             │
└─────────────────────────────────────────────────────────────┘
                            │
                            ▼
                          [...]
"""

print(diagram_2)

print("\n" + "=" * 60)
print("KEY INSIGHTS")
print("=" * 60)
print("""
✓ Each iteration is self-contained
✓ Observations from iteration N inform thought in iteration N+1
✓ State grows with each cycle
✓ Reasoning is grounded in actual observations, not assumptions
""")

## 7.3 Reasoning vs State Interaction

How reasoning artifacts flow into and out of state:

In [ ]:
print_section_header("DIAGRAM 3: Reasoning ↔ State Interaction")

diagram_3 = """
REASONING AND STATE: A Two-Way Flow

┌─────────────────────────────────────────────────────────────┐
│                      AGENT STATE                            │
│                                                             │
│  ┌─────────────────────────────────────────────────┐       │
│  │  Data Fields:                                   │       │
│  │  • goal: "Find papers on transformers"          │       │
│  │  • papers_found: [...]                          │       │
│  │  • selected_papers: [...]                       │       │
│  │  • final_summary: None                          │       │
│  └─────────────────────────────────────────────────┘       │
│                                                             │
│  ┌─────────────────────────────────────────────────┐       │
│  │  Reasoning Artifacts:                           │       │
│  │  • reasoning_trace: [                           │       │
│  │      {thought: "...", action: "...", obs: "..."}│       │
│  │      {thought: "...", action: "...", obs: "..."}│       │
│  │    ]                                            │       │
│  │  • current_step: 3                              │       │
│  └─────────────────────────────────────────────────┘       │
│                                                             │
└──────────────┬──────────────────────────┬──────────────────┘
               │                          │
               │ READ                     │ WRITE
               │ (Input to reasoning)     │ (Output from reasoning)
               │                          │
               ▼                          ▲
         ┌─────────────────────────────────────┐
         │    REASONING ENGINE                 │
         │                                     │
         │  Reads:                             │
         │  • Current state data               │
         │  • Previous reasoning steps         │
         │  • Observations from last action    │
         │                                     │
         │  Decides:                           │
         │  • What to do next                  │
         │  • Which action to take             │
         │                                     │
         │  Writes:                            │
         │  • New reasoning step               │
         │  • Action results                   │
         │  • Updated data fields              │
         │                                     │
         └─────────────────────────────────────┘

FLOW:
1. Reasoning reads current state
2. Reasoning decides on action
3. Action executes (external world interaction)
4. Observation recorded
5. State updated with new data + reasoning artifact
6. Loop repeats
"""

print(diagram_3)

print("\n" + "=" * 60)
print("KEY INSIGHTS")
print("=" * 60)
print("""
✓ State is both INPUT and OUTPUT of reasoning
✓ Reasoning artifacts are stored IN state (not separate)
✓ Each iteration enriches state with:
  - New data (observations, results)
  - New reasoning (thoughts, decisions)
✓ This creates an auditable trail
""")

## 7.4 Single-Agent vs Multi-Agent Reasoning

Comparing reasoning patterns:

In [ ]:
print_section_header("DIAGRAM 4: Single-Agent vs Multi-Agent Reasoning")

diagram_4 = """
┌─────────────────────────────────────────────────────────────┐
│               SINGLE-AGENT REASONING                        │
└─────────────────────────────────────────────────────────────┘

    State
      │
      ▼
  ┌─────────┐
  │ Agent   │
  │ Thinks  │──→ Action ──→ Observation
  └────┬────┘                    │
       │                         │
       └───── Update State ◄─────┘
                │
                ▼
              [Repeat]

• Single reasoning loop
• One perspective
• Linear progression


┌─────────────────────────────────────────────────────────────┐
│              MULTI-AGENT REASONING                          │
└─────────────────────────────────────────────────────────────┘

         Shared State
              │
      ┌───────┴───────┐
      │               │
      ▼               ▼
  ┌─────────┐    ┌─────────┐
  │Agent A  │    │Agent B  │
  │(Critic) │    │(Writer) │
  └────┬────┘    └────┬────┘
       │              │
       │ Reasoning A  │ Reasoning B
       │              │
       ▼              ▼
   Action A       Action B
       │              │
       ▼              ▼
  Obs A           Obs B
       │              │
       └──────┬───────┘
              │
              ▼
      Update Shared State
       (both reasoning traces)
              │
              ▼
           [Repeat]

• Multiple reasoning loops
• Different perspectives
• Potential conflicts
• Need for coordination


┌─────────────────────────────────────────────────────────────┐
│          KEY DIFFERENCE: REASONING CONTRACTS                │
└─────────────────────────────────────────────────────────────┘

Single-Agent:
  • Agent owns all reasoning
  • No conflicts possible
  • Simple validation

Multi-Agent:
  • Must share reasoning artifacts
  • Can have contradictions
  • Requires reasoning alignment checks
  • Need handoff protocols
"""

print(diagram_4)

print("\n" + "=" * 60)
print("KEY INSIGHTS")
print("=" * 60)
print("""
✓ Multi-agent systems need REASONING CONTRACTS:
  - What reasoning does each agent contribute?
  - How do agents hand off work?
  - How are contradictions resolved?

✓ Without shared reasoning artifacts:
  - Agent B can't understand why Agent A did something
  - Debugging multi-agent failures is impossible
  - Coordination breaks down
""")

## 🎯 Key Takeaway: Section 7

> **Visual diagrams reveal the structure of reasoning: it's an iterative loop where thought, action, and observation flow into state, creating an auditable trail. Multi-agent systems require reasoning contracts to coordinate perspectives and avoid conflicts.**

---

<a id="section-8"></a>
# Section 8: Reasoning in Multi-Agent Systems (MAS)

Multi-agent systems require explicit reasoning coordination.

---

## 8.1 Why MAS Needs Reasoning Contracts

In multi-agent systems, agents must:
- **Share reasoning context**: Agent B needs to understand why Agent A made a decision
- **Coordinate work**: Avoid duplicate effort or contradictory actions
- **Resolve conflicts**: When agents disagree, reasoning explains why

**Without reasoning contracts:**
- ❌ Agents work in silos
- ❌ Contradictions go undetected
- ❌ Debugging is impossible
- ❌ Coordination breaks down

---

## 8.2 Multi-Agent State Schema

In [ ]:
@dataclass
class AgentReasoningEntry:
    """Reasoning from a specific agent"""
    agent_name: str
    step_number: int
    thought: str
    action: str
    observation: str
    timestamp: str = field(default_factory=lambda: datetime.now().isoformat())

@dataclass
class MultiAgentState:
    """Shared state for multiple agents"""
    goal: str
    current_agent: Optional[str] = None
    reasoning_history: List[AgentReasoningEntry] = field(default_factory=list)
    draft_content: Optional[str] = None
    critique: Optional[str] = None
    final_content: Optional[str] = None
    is_complete: bool = False
    
    def add_agent_reasoning(self, agent_name: str, thought: str, action: str, observation: str):
        """Add reasoning from a specific agent"""
        entry = AgentReasoningEntry(
            agent_name=agent_name,
            step_number=len(self.reasoning_history) + 1,
            thought=thought,
            action=action,
            observation=observation
        )
        self.reasoning_history.append(entry)
        self.current_agent = agent_name
        return entry
    
    def get_reasoning_by_agent(self, agent_name: str) -> List[AgentReasoningEntry]:
        """Get all reasoning from a specific agent"""
        return [r for r in self.reasoning_history if r.agent_name == agent_name]

print("✓ Multi-agent state schema defined")

## 8.3 Implementing Writer-Critic Multi-Agent System

Two agents with different reasoning roles:

In [ ]:
def writer_agent(state: MultiAgentState) -> MultiAgentState:
    """Agent that writes content"""
    # THOUGHT: What's my role?
    thought = f"My goal is to write content about: {state.goal}. I will create a draft."
    
    # ACTION: Generate draft
    action = "write_draft"
    draft = f"Draft content about {state.goal}: Transformers revolutionized NLP by introducing self-attention mechanisms..."
    
    # OBSERVATION: What did I produce?
    observation = f"Created draft with {len(draft)} characters"
    
    # Record reasoning
    state.add_agent_reasoning("Writer", thought, action, observation)
    state.draft_content = draft
    
    return state

def critic_agent(state: MultiAgentState) -> MultiAgentState:
    """Agent that critiques content"""
    # THOUGHT: What's my role?
    thought = f"I need to review the Writer's draft and provide constructive feedback."
    
    # Check if draft exists
    if not state.draft_content:
        observation = "ERROR: No draft to critique"
        state.add_agent_reasoning("Critic", thought, "check_draft", observation)
        return state
    
    # ACTION: Critique the draft
    action = "critique_draft"
    
    # Analyze the draft
    draft_length = len(state.draft_content)
    has_specific_details = "self-attention" in state.draft_content.lower()
    
    if draft_length < 50:
        critique = "FEEDBACK: Draft is too short. Add more details."
    elif not has_specific_details:
        critique = "FEEDBACK: Draft lacks technical depth. Add specifics about mechanisms."
    else:
        critique = "FEEDBACK: Draft looks good. Could add examples for clarity."
    
    # OBSERVATION: What's my verdict?
    observation = f"Critique completed: {critique}"
    
    # Record reasoning
    state.add_agent_reasoning("Critic", thought, action, observation)
    state.critique = critique
    
    return state

def writer_revise(state: MultiAgentState) -> MultiAgentState:
    """Writer revises based on critique"""
    # THOUGHT: What did the Critic say?
    thought = f"The Critic said: '{state.critique}'. I will revise accordingly."
    
    # ACTION: Revise draft
    action = "revise_draft"
    
    # Incorporate feedback
    if "examples" in state.critique.lower():
        revised = state.draft_content + " For example, BERT uses bidirectional attention for context understanding."
    else:
        revised = state.draft_content + " (Revised based on feedback)"
    
    # OBSERVATION: What's the result?
    observation = "Revision complete"
    
    # Record reasoning
    state.add_agent_reasoning("Writer", thought, action, observation)
    state.final_content = revised
    state.is_complete = True
    
    return state

print("✓ Multi-agent functions defined (Writer, Critic, Writer-Revise)")

## 8.4 Running the Multi-Agent System

In [ ]:
print_section_header("MULTI-AGENT SYSTEM EXECUTION")

# Initialize shared state
mas_state = MultiAgentState(goal="Explain transformers")

print(f"Goal: {mas_state.goal}\n")

# Step 1: Writer creates draft
print("▶ Step 1: Writer Agent")
mas_state = writer_agent(mas_state)
latest = mas_state.reasoning_history[-1]
print(f"  Agent: {latest.agent_name}")
print(f"  Thought: {latest.thought}")
print(f"  Action: {latest.action}")
print(f"  Observation: {latest.observation}")
print(f"  Output: Draft created ({len(mas_state.draft_content)} chars)")

# Step 2: Critic reviews draft
print("\n▶ Step 2: Critic Agent")
mas_state = critic_agent(mas_state)
latest = mas_state.reasoning_history[-1]
print(f"  Agent: {latest.agent_name}")
print(f"  Thought: {latest.thought}")
print(f"  Action: {latest.action}")
print(f"  Observation: {latest.observation}")
print(f"  Critique: {mas_state.critique}")

# Step 3: Writer revises based on critique
print("\n▶ Step 3: Writer Revises")
mas_state = writer_revise(mas_state)
latest = mas_state.reasoning_history[-1]
print(f"  Agent: {latest.agent_name}")
print(f"  Thought: {latest.thought}")
print(f"  Action: {latest.action}")
print(f"  Observation: {latest.observation}")

print(f"\n{'=' * 60}")
print("FINAL RESULT")
print('=' * 60)
print(f"Complete: {mas_state.is_complete}")
print(f"Final content: {mas_state.final_content}")
print(f"\nTotal reasoning steps: {len(mas_state.reasoning_history)}")
print(f"Writer steps: {len(mas_state.get_reasoning_by_agent('Writer'))}")
print(f"Critic steps: {len(mas_state.get_reasoning_by_agent('Critic'))}")

## 8.5 Validating Multi-Agent Reasoning

Check for reasoning alignment and detect contradictions:

In [ ]:
def validate_mas_reasoning_alignment(state: MultiAgentState) -> bool:
    """Validate that agents worked together coherently"""
    
    # Check 1: Writer created draft before Critic reviewed
    writer_steps = state.get_reasoning_by_agent("Writer")
    critic_steps = state.get_reasoning_by_agent("Critic")
    
    if critic_steps and not writer_steps:
        raise ReasoningValidationError(
            "Critic acted without Writer creating content first"
        )
    
    if critic_steps:
        first_critic_step = min(c.step_number for c in critic_steps)
        first_writer_step = min(w.step_number for w in writer_steps)
        if first_critic_step < first_writer_step:
            raise ReasoningValidationError(
                "Critic acted before Writer (ordering violation)"
            )
    
    print("✓ Agent ordering validated: Writer → Critic → Writer")
    
    # Check 2: Critique was actually used
    if state.critique and len(writer_steps) > 1:
        revision_step = writer_steps[-1]
        if state.critique not in revision_step.thought:
            raise ReasoningValidationError(
                "Writer revision doesn't reference Critic's feedback"
            )
    print("✓ Reasoning handoff validated: Writer acknowledged Critic's feedback")
    
    # Check 3: No contradictions in reasoning
    for entry in state.reasoning_history:
        if "ERROR" in entry.observation and state.is_complete:
            raise ReasoningValidationError(
                f"Agent {entry.agent_name} encountered error but system marked complete"
            )
    print("✓ No contradictions detected in reasoning history")
    
    return True

def validate_mas_no_contradictions(state: MultiAgentState) -> bool:
    """Check for logical contradictions between agents"""
    
    # Example: If Critic says "too short" but Writer says "looks complete"
    critic_feedback = state.critique if state.critique else ""
    
    if "too short" in critic_feedback.lower() and state.is_complete:
        # Check if Writer actually addressed this
        writer_revisions = [r for r in state.reasoning_history 
                           if r.agent_name == "Writer" and "revise" in r.action.lower()]
        if not writer_revisions:
            raise ReasoningValidationError(
                "Critic said content too short, but Writer didn't revise"
            )
    
    print("✓ No logical contradictions between agents")
    return True

print_section_header("VALIDATING MULTI-AGENT REASONING")

try:
    validate_mas_reasoning_alignment(mas_state)
    validate_mas_no_contradictions(mas_state)
    
    print("\n" + "=" * 60)
    print("✅ MULTI-AGENT REASONING VALIDATED")
    print("=" * 60)
    print("The agents:")
    print("  • Worked in proper order")
    print("  • Shared reasoning context")
    print("  • No contradictions detected")
    print("  • Proper handoff of work")
    
except ReasoningValidationError as e:
    print(f"\n❌ VALIDATION FAILED: {e}")

## 8.6 Failure Mode: No Shared Reasoning

What happens without shared reasoning artifacts?

In [ ]:
print_section_header("FAILURE MODE: Isolated Agent Reasoning")

# Create broken MAS where agents don't share reasoning
broken_mas = MultiAgentState(goal="Write article")

# Writer works in isolation (no reasoning recorded)
broken_mas.draft_content = "Some content..."
# Note: NO reasoning added to state

# Critic works in isolation (no reasoning recorded)
broken_mas.critique = "Needs work"
# Note: NO reasoning added to state

# Final content appears (no reasoning about revision)
broken_mas.final_content = "Different content..."
broken_mas.is_complete = True

print("Broken MAS created (no shared reasoning)")
print(f"Draft: {broken_mas.draft_content}")
print(f"Critique: {broken_mas.critique}")
print(f"Final: {broken_mas.final_content}")
print(f"Reasoning history: {len(broken_mas.reasoning_history)} entries")

print("\n" + "-" * 60)
print("Attempting validation...")
print("-" * 60)

try:
    validate_mas_reasoning_alignment(broken_mas)
    print("✅ Validation passed (this should NOT happen)")
except ReasoningValidationError as e:
    print(f"❌ Validation caught the error!")
    print(f"   Error: {e}")
    print("\n✓ This is GOOD - validation detected missing reasoning")

print("\n" + "=" * 60)
print("WHY THIS FAILS")
print("=" * 60)
print("""
Without shared reasoning artifacts:

❌ Can't verify Writer created draft before Critic reviewed
❌ Can't verify Critic actually reviewed the draft
❌ Can't verify Writer used Critic's feedback
❌ Can't debug coordination issues
❌ Can't audit the decision-making process

This is why reasoning contracts are critical in MAS.
""")

## 🎯 Key Takeaway: Section 8

> **Multi-agent systems require reasoning contracts: shared state where each agent records its thoughts, actions, and observations. Without explicit reasoning handoffs, agents work in silos, contradictions go undetected, and coordination breaks down.**

---

<a id="section-9"></a>
# Section 9: Common Reasoning Failures

Let's examine the most common mistakes in agent reasoning.

---

## 9.1 Failure Mode 1: Prompt-Only Reasoning

**The Mistake:** Putting all reasoning in the prompt and hoping the LLM will "just do it."

In [ ]:
print_section_header("FAILURE 1: Prompt-Only Reasoning")

# BAD: All reasoning in prompt
bad_prompt = """
You are a research agent. 
1. Search for papers on transformers
2. Filter to the top 3 by citations
3. Read their abstracts
4. Synthesize a summary

Please complete this task.
"""

print("❌ BAD APPROACH:")
print(bad_prompt)

print("\n" + "-" * 60)
print("What happens:")
print("-" * 60)
print("""
• LLM generates text that LOOKS like it did those steps
• But no actual search occurred
• No actual papers were retrieved
• Summary is hallucinated
• No way to verify any step

Result: "Based on recent papers, transformers use attention..."
  ↑ This is made up!
""")

print("\n" + "-" * 60)
print("WHY THIS FAILS:")
print("-" * 60)
print("""
1. No actual tool calls (search didn't happen)
2. No state tracking (can't verify papers exist)
3. No observable reasoning (can't debug)
4. No validation possible (no evidence to check)

The LLM generates plausible text, but does no actual work.
""")

## 9.2 Failure Mode 2: Hidden Chain-of-Thought

**The Mistake:** Using chain-of-thought prompting but not externalizing reasoning into state.

In [ ]:
print_section_header("FAILURE 2: Hidden Chain-of-Thought")

# BAD: Chain-of-thought in response text
simulated_response = """
Let me think step by step:

Step 1: I would search for papers on transformers
Step 2: I would filter by citation count
Step 3: I would read the top papers
Step 4: Based on my analysis, transformers work by...

Final answer: Transformers use self-attention mechanisms.
"""

print("❌ BAD APPROACH:")
print(simulated_response)

print("\n" + "-" * 60)
print("What's in state after this:")
print("-" * 60)
print("""
state = {
    "response": "Let me think step by step:..."
}

That's it. The "reasoning" is just unstructured text.
""")

print("\n" + "-" * 60)
print("WHY THIS FAILS:")
print("-" * 60)
print("""
1. Reasoning is in TEXT, not STRUCTURED STATE
   → Can't programmatically validate steps
   
2. Can't branch based on intermediate results
   → "If step 2 finds <5 papers, do X else Y" is impossible
   
3. Can't extract intermediate values
   → Can't access the "list of papers" for further processing
   
4. Can't call tools between steps
   → All "steps" are simulated, not actual actions
   
5. Reasoning and data are mixed
   → Can't separate "what was decided" from "what was found"

The reasoning LOOKS explicit, but it's still hidden in text.
""")

## 9.3 Failure Mode 3: Over-Trusting LLM "Confidence"

**The Mistake:** Assuming that confident-sounding responses are accurate.

In [ ]:
print_section_header("FAILURE 3: Over-Trusting LLM Confidence")

# Simulated confident but wrong LLM response
confident_hallucination = """
Based on my comprehensive analysis of recent literature,
I can confidently state that transformers were invented
in 2019 by researchers at OpenAI. The architecture uses
a novel "circular attention" mechanism that processes
sequences in both forward and backward directions 
simultaneously, achieving 99.8% accuracy on all NLP tasks.

Sources:
- "Transformers Unleashed" (Smith et al., 2019)
- "The Circular Attention Paper" (Jones, 2020)
"""

print("❌ LLM Response (sounds confident!):")
print(confident_hallucination)

print("\n" + "-" * 60)
print("What's ACTUALLY true:")
print("-" * 60)
print("""
✗ Transformers were invented in 2017, not 2019
✗ By Google (Vaswani et al.), not OpenAI
✗ "Circular attention" doesn't exist (made up)
✗ Those papers don't exist (hallucinated citations)
✗ 99.8% accuracy on all tasks is absurd

But it SOUNDS authoritative!
""")

print("\n" + "-" * 60)
print("WHY THIS FAILS:")
print("-" * 60)
print("""
1. LLMs generate confident-sounding text even when wrong
   → Confidence ≠ Accuracy
   
2. No source verification
   → Can't check if cited papers exist
   
3. No grounding in evidence
   → No actual papers retrieved from database
   
4. No validation layer
   → No fact-checking against known sources

FIX: Never trust LLM output without validation
     Always require evidence from state (retrieved docs, API calls, etc.)
""")

## 9.4 Failure Mode 4: Reasoning Not Grounded in Evidence

**The Mistake:** Making inferences or conclusions without retrievable evidence in state.

In [ ]:
print_section_header("FAILURE 4: Reasoning Without Evidence")

# Simulated state with conclusion but no evidence
ungrounded_state = {
    "goal": "Analyze transformer performance",
    "reasoning": "Based on extensive analysis...",
    "conclusion": "Transformers outperform RNNs by 40% on all tasks",
    # NOTE: No evidence!
    "papers_retrieved": [],  # Empty!
    "benchmarks_run": [],    # Empty!
    "data_analyzed": None    # None!
}

print("❌ BAD STATE:")
pprint(ungrounded_state)

print("\n" + "-" * 60)
print("The Problem:")
print("-" * 60)
print("""
The agent claims "transformers outperform RNNs by 40%"

But where's the evidence?
✗ No papers in state
✗ No benchmarks in state
✗ No data in state

The conclusion is ungrounded - it's a hallucination.
""")

print("\n" + "-" * 60)
print("WHY THIS FAILS:")
print("-" * 60)
print("""
1. Conclusion without evidence
   → Can't verify the 40% claim
   
2. No traceable reasoning path
   → Can't see how conclusion was reached
   
3. Can't audit the analysis
   → No raw data to check
   
4. Can't reproduce the result
   → Missing all inputs

FIX: Every conclusion MUST reference evidence in state
     Validation should check: evidence exists → conclusion drawn
""")

print("\n" + "-" * 60)
print("CORRECT APPROACH:")
print("-" * 60)

grounded_state = {
    "goal": "Analyze transformer performance",
    "papers_retrieved": [
        {"title": "Paper A", "result": "35% improvement"},
        {"title": "Paper B", "result": "42% improvement"}
    ],
    "reasoning": "Averaging results from Paper A (35%) and Paper B (42%)",
    "conclusion": "Transformers outperform RNNs by ~38.5% based on 2 studies"
}

print("✅ GOOD STATE:")
pprint(grounded_state)
print("\nNow we can:")
print("✓ Verify the papers exist")
print("✓ Check the math (35+42)/2 = 38.5")
print("✓ Trace reasoning from evidence to conclusion")
print("✓ Audit the quality of sources")

## 9.5 Summary of Common Failures

| Failure Mode | Symptom | Fix |
|--------------|---------|-----|
| **Prompt-Only** | No actual work done, just text | Implement actual tool calls + state |
| **Hidden CoT** | Reasoning in text, not structure | Externalize to typed state fields |
| **Over-Trust** | Confident hallucinations | Always validate against evidence |
| **Ungrounded** | Conclusions without evidence | Require evidence → reasoning → conclusion |

---

## 🎯 Key Takeaway: Section 9

> **Common reasoning failures all stem from the same root cause: treating reasoning as text generation instead of as a structured, evidence-based, verifiable process. The fix is always the same: externalize reasoning into state, ground it in evidence, and validate rigorously.**

---

<a id="section-10"></a>
# Section 10: Final Mental Model & Production Checklist

Bringing it all together.

---

## 10.1 The Complete Mental Model of Agent Reasoning

### Core Principles

```
┌─────────────────────────────────────────────────────────────┐
│          AGENT REASONING MENTAL MODEL                       │
└─────────────────────────────────────────────────────────────┘

1. REASONING IS A PROCESS, NOT A PROMPT
   • Not: "LLM, think about this"
   • Yes: Iterative loop of thought → action → observation

2. REASONING MUST BE EXTERNALIZED
   • Not: Hidden in LLM's internal chain-of-thought
   • Yes: Stored in structured state as typed fields

3. REASONING MUST BE GROUNDED IN EVIDENCE
   • Not: LLM generates conclusions from memory
   • Yes: Conclusions derived from retrieved evidence in state

4. REASONING MUST BE VALIDATABLE
   • Not: Trust the output blindly
   • Yes: Programmatic checks ensure correctness

5. REASONING IS STATE EVOLUTION
   • Not: Stateless prompt → response
   • Yes: State grows with each reasoning step
```

---

## 10.2 The Three Layers of Agent Reasoning

In [ ]:
print_section_header("The Three Layers of Reasoning")

three_layers = """
┌─────────────────────────────────────────────────────────────┐
│                    LAYER 1: DATA                            │
│  What the agent knows                                       │
├─────────────────────────────────────────────────────────────┤
│  • Retrieved documents                                      │
│  • API responses                                            │
│  • User inputs                                              │
│  • Calculated results                                       │
│  • Observations from actions                                │
└─────────────────────────────────────────────────────────────┘
                            ▲
                            │ Feeds into
                            │
┌─────────────────────────────────────────────────────────────┐
│                  LAYER 2: REASONING                         │
│  How the agent decides                                      │
├─────────────────────────────────────────────────────────────┤
│  • Thoughts (what to do next)                               │
│  • Decisions (which action to take)                         │
│  • Justifications (why this action)                         │
│  • Reasoning trace (history of decisions)                   │
└─────────────────────────────────────────────────────────────┘
                            ▲
                            │ Evaluated by
                            │
┌─────────────────────────────────────────────────────────────┐
│                 LAYER 3: VALIDATION                         │
│  How we verify correctness                                  │
├─────────────────────────────────────────────────────────────┤
│  • Sequence checks (steps in order)                         │
│  • Evidence checks (data exists)                            │
│  • Consistency checks (no contradictions)                   │
│  • Completeness checks (all steps present)                  │
└─────────────────────────────────────────────────────────────┘

KEY INSIGHT:
All three layers are STORED IN STATE.
You can inspect, debug, and audit all three at any time.
"""

print(three_layers)

print("\n" + "=" * 60)
print("What This Means in Practice")
print("=" * 60)
print("""
❌ DON'T: Mix layers
   • Don't put reasoning in prompts
   • Don't hide data in LLM context
   • Don't skip validation

✅ DO: Separate concerns
   • Data goes in state.data_fields
   • Reasoning goes in state.reasoning_trace
   • Validation checks both layers
""")

## 10.3 Production-Ready Reasoning Checklist

Use this checklist when designing or reviewing agent reasoning:

In [ ]:
print_section_header("PRODUCTION REASONING CHECKLIST")

checklist = """
☑️ AGENT REASONING DESIGN CHECKLIST
════════════════════════════════════════════════════════════

ARCHITECTURE
────────────────────────────────────────────────────────────
 ☐  1. Reasoning is externalized into structured state
 ☐  2. State schema includes reasoning_trace field
 ☐  3. Each reasoning step has: thought, action, observation
 ☐  4. Reasoning artifacts are typed (TypedDict/dataclass)
 ☐  5. Clear separation between data and reasoning

ITERATION
────────────────────────────────────────────────────────────
 ☐  6. Reasoning follows Thought → Action → Observation loop
 ☐  7. Each iteration updates state (not just returns text)
 ☐  8. Observations from step N feed into thought for step N+1
 ☐  9. Clear termination condition (when to stop reasoning)
 ☐ 10. Maximum iteration limit (prevent infinite loops)

EVIDENCE
────────────────────────────────────────────────────────────
 ☐ 11. All conclusions reference evidence in state
 ☐ 12. Evidence is retrieved from actual sources (not LLM memory)
 ☐ 13. Documents/data stored in state, not just cited
 ☐ 14. Can trace from evidence → reasoning → conclusion
 ☐ 15. No hallucinated references or data

VALIDATION
────────────────────────────────────────────────────────────
 ☐ 16. Validation functions check reasoning sequence
 ☐ 17. Verify no steps were skipped
 ☐ 18. Verify evidence exists before conclusions
 ☐ 19. Verify observations are non-empty
 ☐ 20. Automated tests for validation logic

DEBUGGING & OBSERVABILITY
────────────────────────────────────────────────────────────
 ☐ 21. Can print/log complete reasoning trace
 ☐ 22. Each step has timestamp for debugging
 ☐ 23. Can replay reasoning for audit
 ☐ 24. Errors include reasoning context
 ☐ 25. Monitoring for reasoning quality metrics

MULTI-AGENT (if applicable)
────────────────────────────────────────────────────────────
 ☐ 26. Shared state includes agent_name for each step
 ☐ 27. Clear reasoning contracts between agents
 ☐ 28. Validation checks for agent coordination
 ☐ 29. Contradiction detection between agents
 ☐ 30. Reasoning handoff is explicit in state

PRODUCTION SCALING
────────────────────────────────────────────────────────────
 ☐ 31. Reasoning trace has size limits (prevent memory bloat)
 ☐ 32. State serialization/deserialization works
 ☐ 33. Can resume from partial reasoning state
 ☐ 34. Reasoning steps are idempotent (safe to retry)
 ☐ 35. Performance metrics: steps per task, time per step
"""

print(checklist)

print("\n" + "=" * 60)
print("HOW TO USE THIS CHECKLIST")
print("=" * 60)
print("""
1. Before implementing: Review checklist, design state schema
2. During development: Check off items as you implement
3. Before deployment: All items should be ✓
4. In production: Monitor items 31-35 continuously

If you can check all boxes, your reasoning is production-ready.
""")

## 10.4 Scaling Reasoning in Production

Practical guidance for production deployments:

In [ ]:
print_section_header("SCALING REASONING IN PRODUCTION")

scaling_guide = """
CHALLENGE 1: REASONING TRACE SIZE
──────────────────────────────────────────────────────────

Problem: Long-running agents accumulate large reasoning traces
Solution:
  • Set max_reasoning_steps limit
  • Implement trace summarization (keep summaries, discard details)
  • Prune intermediate steps after validation passes
  • Store full trace in separate logging system

Example:
  if len(state.reasoning_trace) > MAX_STEPS:
      # Keep first 5, last 10, summarize middle
      state.reasoning_trace = (
          state.reasoning_trace[:5] + 
          [summarize(state.reasoning_trace[5:-10])] +
          state.reasoning_trace[-10:]
      )


CHALLENGE 2: VALIDATION OVERHEAD
──────────────────────────────────────────────────────────

Problem: Running validation after every step is expensive
Solution:
  • Validate only at checkpoints (every N steps)
  • Use lightweight checks during iteration
  • Full validation only before final output
  • Cache validation results

Example:
  if step_num % VALIDATION_INTERVAL == 0:
      validate_reasoning_so_far(state)
  
  # Always validate before returning to user
  if is_complete:
      full_validation(state)


CHALLENGE 3: MULTI-AGENT COORDINATION AT SCALE
──────────────────────────────────────────────────────────

Problem: Many agents → many reasoning traces → hard to debug
Solution:
  • Each agent has clear responsibility contract
  • Reasoning includes agent_id in every step
  • Centralized coordination logic
  • Agent-specific validation rules

Example:
  state.add_reasoning(
      agent_id="researcher_42",
      phase="search",
      ...
  )
  
  # Can filter: show only researcher_42's reasoning
  agent_trace = [s for s in state.reasoning_trace 
                 if s.agent_id == "researcher_42"]


CHALLENGE 4: REASONING OBSERVABILITY
──────────────────────────────────────────────────────────

Problem: Need to monitor reasoning quality in production
Solution:
  • Log reasoning metrics: steps_per_task, avg_time_per_step
  • Alert on anomalies: too many steps, stuck loops
  • Sample reasoning traces for human review
  • A/B test different reasoning strategies

Metrics to track:
  • Average steps to completion
  • Validation pass rate
  • Evidence retrieval success rate
  • Reasoning contradiction rate (multi-agent)
  • User satisfaction with reasoning transparency


CHALLENGE 5: RESUMABILITY
──────────────────────────────────────────────────────────

Problem: Long tasks may need to pause/resume
Solution:
  • State must be fully serializable
  • Save state after each major step
  • On resume, validate partial state
  • Ensure idempotent reasoning steps

Example:
  # Save checkpoint
  save_state_to_db(state)
  
  # Resume later
  state = load_state_from_db(task_id)
  validate_partial_state(state)
  continue_reasoning(state)
"""

print(scaling_guide)

print("\n" + "=" * 60)
print("PRODUCTION DEPLOYMENT PATTERN")
print("=" * 60)
print("""
1. Design reasoning schema (types, validation)
2. Implement reasoning loop (thought → action → observation)
3. Add validation layer
4. Add observability (logging, metrics)
5. Load test with reasoning trace limits
6. Deploy with monitoring
7. Iterate based on production metrics

Remember: Reasoning is infrastructure, not an afterthought.
""")

## 10.5 Final Summary

### What We've Learned

Throughout this notebook, we've built a complete understanding of agent reasoning:

1. **Sections 1-3:** Conceptual foundations
   - Reasoning is externalized, structured decision-making
   - Not chain-of-thought in prompts
   - Iterative process: Thought → Action → Observation

2. **Sections 4-6:** Implementation
   - Single-agent reasoning with explicit state
   - Validation frameworks to prevent hallucination
   - RAG agents with evidence-based reasoning

3. **Sections 7-8:** Advanced patterns
   - Visual reasoning flows
   - Multi-agent coordination with reasoning contracts

4. **Sections 9-10:** Production readiness
   - Common failure modes and how to avoid them
   - Production checklist and scaling guidance

---

## 🎯 Final Takeaway

> **Agent reasoning is the structured, externalized, evidence-based, iterative process by which agents decide what to do. It must be stored in state, validated rigorously, and designed with production scaling in mind. Reasoning is not what you prompt the LLM to do—it's the infrastructure you build around the LLM.**

---

## Next Steps

1. **Practice:** Implement a simple agent using the patterns in this notebook
2. **Validate:** Use the checklist to review your reasoning design
3. **Scale:** Apply the production guidance when deploying
4. **Iterate:** Monitor reasoning metrics and improve

**Remember:** Good reasoning design is what separates demos from production systems.

---

# End of Notebook

Thank you for working through **Agent Reasoning From First Principles**!

For questions or to dive deeper into specific patterns, revisit the relevant sections.